# Local interpretability

This notebook will evaluate the contribution of each feature towards the target value, for a single observation, using StatsModels.

Based on [Christoph Molnar's book](https://christophm.github.io/interpretable-ml-book/limo.html#effect-plot)

In [ ]:
# Packages

import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler